In [2]:
import requests
import pandas as pd
import datetime as dt
import numpy as np
import random as rd

In [3]:
matches = pd.read_csv('matches.csv')
pd.set_option('display.max_columns', None)

In [ ]:
#Sends request to get past matches

def pastMatchesRequest(date_from, date_to, competition_id, page):
    url = "https://live-score-api.p.rapidapi.com/scores/history.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'from': date_from,
        'to': date_to,
        'competition_id': competition_id,
        'page': page
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [ ]:
#Sends request to get stats from a specific match

def matchStatsRequest(match_id):
    url = "https://live-score-api.p.rapidapi.com/matches/stats.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'match_id': match_id
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [392]:
#Stores information from 'Past matches' on a list

def addMatchInfo(response, continent, country, league_name, type, tier):
    for info in response['data']['match']:
        if(info['ht_score'] == ''):
            info['ht_score'] = '0 - 0'
            
        if(info['scheduled'] is None or info['scheduled'] == ""):
            matches_list.append([
                info['id'],
                info['home_id'],
                info['home_name'],
                info['away_id'],
                info['away_name'],
                info['score'].split(' -')[0],
                info['score'].split('- ')[1],
                info['ht_score'].split(' -')[0],
                info['ht_score'].split('- ')[1],
                info['location'],
                'City',
                country,
                continent,
                league_name,
                type,
                tier,
                np.nan,
                np.nan,
                info['date'].split('-')[2],
                dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
                info['date'].split('-')[1],
                info['date'].split('-')[0],
                'F'
            ])
        else:
            matches_list.append([
                info['id'],
                info['home_id'],
                info['home_name'],
                info['away_id'],
                info['away_name'],
                info['score'].split(' -')[0],
                info['score'].split('- ')[1],
                info['ht_score'].split(' -')[0],
                info['ht_score'].split('- ')[1],
                info['location'],
                'City',
                country,
                continent,
                league_name,
                type,
                tier,
                info['scheduled'].split(':')[0],
                info['scheduled'].split(':')[1],
                info['date'].split('-')[2],
                dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
                info['date'].split('-')[1],
                info['date'].split('-')[0],
                'F'
            ])

In [ ]:
#Updates df with 'Match stats' information

def addMatchStats(rowIndex, matchId):
    response = matchStatsRequest(matchId)
    if(response['success'] == False):
        return False
    if(response['data'] != []):
        matches.at[rowIndex,'home_yc'] = response['data']['yellow_cards'].split(':')[0]
        matches.at[rowIndex,'away_yc'] = response['data']['yellow_cards'].split(':')[1]
        matches.at[rowIndex,'home_rc'] = response['data']['red_cards'].split(':')[0]
        matches.at[rowIndex,'away_rc'] = response['data']['red_cards'].split(':')[1]
        if(response['data']['possesion'] is None):
            matches.at[rowIndex,'home_possession'] = np.nan
            matches.at[rowIndex,'away_possession'] = np.nan
        else:
            matches.at[rowIndex,'home_possession'] = response['data']['possesion'].split(':')[0]
            matches.at[rowIndex,'away_possession'] = response['data']['possesion'].split(':')[1]
            
        if(response['data']['attempts_on_goal'] is None):
            matches.at[rowIndex,'home_shots'] = np.nan
            matches.at[rowIndex,'away_shots'] = np.nan 
        else:
            matches.at[rowIndex,'home_shots'] = response['data']['attempts_on_goal'].split(':')[0] 
            matches.at[rowIndex,'away_shots'] = response['data']['attempts_on_goal'].split(':')[1] 
            
        if(response['data']['shots_on_target'] is not None):
            matches.at[rowIndex,'home_shots_on_target'] = response['data']['shots_on_target'].split(':')[0]
            matches.at[rowIndex,'away_shots_on_target'] = response['data']['shots_on_target'].split(':')[1]
        elif(response['data']['attempts_on_goal'] is not None and response['data']['shots_off_target'] is not None):
            matches.at[rowIndex,'home_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[0]) - int(response['data']['shots_off_target'].split(':')[0])
            matches.at[rowIndex,'away_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[1]) - int(response['data']['shots_off_target'].split(':')[1])
        else:
            matches.at[rowIndex,'home_shots_on_target'] = np.nan
            matches.at[rowIndex,'away_shots_on_target'] = np.nan 
       
        if(response['data']['fauls'] is None):
            matches.at[rowIndex,'home_fouls'] = np.nan
            matches.at[rowIndex,'away_fouls'] = np.nan
        else:
            matches.at[rowIndex,'home_fouls'] = response['data']['fauls'].split(':')[0]
            matches.at[rowIndex,'away_fouls'] = response['data']['fauls'].split(':')[1]
        
        if(response['data']['attacks'] is None):
            matches.at[rowIndex,'home_attacks'] = np.nan
            matches.at[rowIndex,'away_attacks'] = np.nan
        else:
            matches.at[rowIndex,'home_attacks'] = response['data']['attacks'].split(':')[0]
            matches.at[rowIndex,'away_attacks'] = response['data']['attacks'].split(':')[1]
    matches.at[rowIndex,'got_stats'] = 'T'
    return True

In [ ]:
#Gets past matches and stores it on a df

def getMatches(date_from, date_to, competition_id, continent, country, league_name, type, tier):
    page = 1
    
    response = pastMatchesRequest(date_from, date_to, competition_id, page)
    addMatchInfo(response, continent, country, league_name, type, tier)
    
    total_pages = response['data']['total_pages']
    page += 1
    
    while(page <= total_pages):
        response = pastMatchesRequest(date_from, date_to, competition_id, page)
        addMatchInfo(response, continent, country, league_name, type, tier)
        page += 1
    
    return pd.DataFrame(matches_list, columns =['match_id','home_id','home_name','away_id','away_name','home_ft_goals','away_ft_goals','home_ht_goals','away_ht_goals','stadium_name','city','country','continent','competition_name','type','tier','hour','minute','day','dayofweek','month','year','got_stats'])

In [ ]:
#Gets match stats from top to bottom until an error is found

def getMatchStats():
    success = True
    
    while(success):
        rowIndex = matches[matches['got_stats'] == 'F'].head(1).tail(1).index
        matchId = matches[matches['got_stats'] == 'F'].head(1).tail(1)['match_id']
        success = addMatchStats(rowIndex, matchId)

In [ ]:
def updateLocation(team_id, team_name, stadium_name, city):
    matches.loc[matches.home_id == team_id, 'home_name'] = team_name   
    matches.loc[matches.away_id == team_id, 'away_name'] = team_name 
    matches.loc[matches.home_id == team_id, 'stadium_name'] = stadium_name
    matches.loc[matches.home_id == team_id, 'city'] = city

In [ ]:
def updateTeamName(old_name, new_name):
    matches.loc[matches.home_name == old_name, 'home_name'] = new_name
    matches.loc[matches.away_name == old_name, 'away_name'] = new_name

In [383]:
def fillStats(column_name):
    df = matches[matches['continent'] == 'South America']
    values = df.groupby('hour').count().index
    frequency = df.groupby('hour').count()['match_id'].values
    rowCount = len(df[df['hour'].notnull()])
    
    while not(matches[matches[column_name].isnull()].empty):
        index = matches[matches[column_name].isnull()].head(1).index
        matches.at[index,column_name] = np.random.choice(values,p = frequency/rowCount)

In [115]:
def fillPossession():
    while not(matches[matches['away_possession'].isnull()].empty):
        row = matches[matches['away_possession'].isnull()].head(1)
        matches.at[row.index,'away_possession'] = 100 - row.home_possession

In [386]:
def fillShots():
    home_values = matches.groupby('home_shots').count().index
    home_frequency = matches.groupby('home_shots').count()['match_id'].values
    
    away_values = matches.groupby('away_shots').count().index
    away_frequency = matches.groupby('away_shots').count()['match_id'].values 
    
    rowCount = len(matches[matches['home_shots'].notnull()])
    
    while not(matches[matches['home_shots'].isnull()].empty):
        row = matches.loc[matches['home_shots'].isnull()].head(1)

        home_shots = np.random.choice(home_values,p = home_frequency/rowCount)
        matches.at[row.index,'home_shots'] = home_shots

        while(True):
            if(home_shots > row.home_ft_goals.values[0]):
                matches.at[row.index,'home_shots'] = home_shots
                break
            home_shots = np.random.choice(home_values,p = home_frequency/rowCount)

        if(home_shots <= 5):
            matches.at[row.index,'home_shots_on_target'] = home_shots - rd.randint(0,home_shots)
        else:
            matches.at[row.index,'home_shots_on_target'] = home_shots - rd.randint(2,home_shots)            

        away_shots = np.random.choice(away_values,p = away_frequency/rowCount)
        matches.at[row.index,'away_shots'] = away_shots

        while(True):
            if(away_shots > row.away_ft_goals.values[0]):
                matches.at[row.index,'away_shots'] = away_shots
                break
            away_shots = np.random.choice(away_values,p = away_frequency/rowCount)

        if(away_shots <= 5):
            matches.at[row.index,'away_shots_on_target'] = away_shots - rd.randint(0,away_shots)
        else:
            matches.at[row.index,'away_shots_on_target'] = away_shots - rd.randint(2,away_shots)    

# Competition Ids

1 - Bundesliga (Both)

93 - 2.Bundesliga (Both)

167 - DFB Pokal (Last)

2 - Premier League (Both)

77 - Championship (Both)

150 - EFL Cup (Last)

152 - FA Cup (Last)

3 - La Liga (Both)

79 - Spanish Segunda Division (Both)

334 - Copa Del Rey (Last)

4 - Serie A (Both)

87 - Serie B (Both)

179 - Coppa Italia (Last)

5 - Ligue 1 (Both)

162 - Coupe de France (Last)

163 - Coupe Ligue (Last)

6 - Turkish Super Lig (Both)

347 - Fortis Turkish Cup (Last) 

7 - Russian Premier League (Both)

311 - Kubok (Last) 

8 - Primeira Liga (Both)

92 - Segunda Liga (Both)

212 - Taça de Portugal (Last)

196 - Eredivisie

9 - Greek Super League (Both)

64 - Ukranian Premier League (Both)

244 - Champions League (Both)
 
245 - Europa League (Both)

24 - Brasileirão (Both)

23 - Liga Professional (Both)

329 - Copa Libertadores (Both)

76 - MLS (Both)

45 - Liga MX (Both)

26 - Chinese Super League (Both)

28 - J-League (Both)

67 - A-League (Last)

42 - Tunisian Ligue 1 (Both)

In [ ]:
#Gets matches from a competitions and stores them in 'df'

matches_list = []
df = getMatches('2019-07-01','2020-12-21', 67, 'Oceania', 'Australia', 'A-League', 'League', 1)
df

In [28]:
#Appends df to matches and stores it in csv file

matches = matches.append(df, ignore_index = True)
matches.to_csv('matches.csv', index=False);

In [ ]:
#Gets match stats until hour limit is reached

getMatchStats()
matches[matches['got_stats'] == 'F']
matches.to_csv('matches.csv', index=False);

In [4]:
matches

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
0,152587,7,Liverpool,492,Norwich City,4,1,0,0,Anfield,Liverpool,England,Europe,Premier League,League,1,19.0,0.0,9,Friday,8,2019,T,1.0,3.0,0.0,1.0,55.0,45.0,7.0,3.0,1.0,1.0,13.0,6.0,69.0,8.0
1,152653,14,West Ham United,12,Manchester City,0,5,0,1,London Stadium,London,England,Europe,Premier League,League,1,11.0,30.0,10,Saturday,8,2019,T,4.0,2.0,0.0,0.0,64.0,36.0,8.0,7.0,1.0,1.0,13.0,13.0,84.0,105.0
2,152730,1,AFC Bournemouth,281,Sheffield United,1,1,0,0,Vitality Stadium,Bournemouth,England,Europe,Premier League,League,1,14.0,0.0,10,Saturday,8,2019,T,1.0,0.0,0.0,0.0,55.0,45.0,20.0,7.0,6.0,5.0,16.0,12.0,102.0,176.0
3,152731,8,Burnley,4,Southampton,3,0,0,0,Turf Moor,Burnley,England,Europe,Premier League,League,1,14.0,0.0,10,Saturday,8,2019,T,1.0,0.0,0.0,0.0,33.0,67.0,9.0,4.0,0.0,4.0,14.0,21.0,113.0,67.0
4,152732,3,Crystal Palace,20,Everton,0,0,0,0,Selhurst Park,London,England,Europe,Premier League,League,1,14.0,0.0,10,Saturday,8,2019,T,1.0,2.0,0.0,0.0,47.0,53.0,12.0,4.0,6.0,0.0,12.0,12.0,73.0,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10008,204837,202,Wellington Phoenix,201,Perth Glory,0,1,0,1,Westpac Stadium,Wellington,Australia,Oceania,A-League,League,1,7.0,0.0,22,Saturday,8,2020,T,1.0,4.0,0.0,0.0,44.0,56.0,10.0,9.0,3.0,4.0,8.0,8.0,116.0,95.0
10009,205208,3185,Brisbane Roar FC,4545,Western United,0,1,0,1,Suncorp Stadium,Brisbane,Australia,Oceania,A-League,League,1,8.0,0.0,23,Sunday,8,2020,T,3.0,1.0,0.0,0.0,57.0,43.0,10.0,9.0,1.0,1.0,21.0,14.0,85.0,82.0
10010,205619,200,Melbourne City FC,4545,Western United,2,0,0,0,AAMI Park,Melbourne,Australia,Oceania,A-League,League,1,7.0,0.0,26,Wednesday,8,2020,T,0.0,0.0,0.0,0.0,45.0,55.0,8.0,6.0,1.0,3.0,1.0,1.0,18.0,19.0
10011,205629,481,Sydney FC,201,Perth Glory,2,0,2,0,Netstrata Jubilee Stadium,Sydney,Australia,Oceania,A-League,League,1,10.0,10.0,26,Wednesday,8,2020,T,0.0,0.0,0.0,0.0,63.0,37.0,2.0,0.0,2.0,0.0,4.0,0.0,24.0,13.0
